In [2]:
import os
import snowflake.snowpark.functions as F

In [61]:
path_utils = os.path.join(os.getcwd(),'Utils','utils.ipynb')
%run $path_utils

####################################################
Available packages:
    from snowflake.snowpark.session import Session
    from snowflake.snowpark import functions as F
    from snowflake.snowpark.types import *

    import configparser
    import os

####################################################

Available functions, for details use help(<function_name>:
    get_snowpark_session
    set_schema
    read_csv_from_stage
    create_hz_dim



In [62]:
session = get_snowpark_session()
set_schema('HZ_CLEAR_STRATEGY')

[Row(status='Statement executed successfully.')]

## DIM_CALENDAR

In [63]:
query_dim_calendar = """
    select distinct
        cast(d.date_of_game as date) as "date",
        year(cast(d.date_of_game as date)) as year,
        month(cast(d.date_of_game as date)) as month,
        day(cast(d.date_of_game as date)) as day,
        cast(to_char(cast(d.date_of_game as date),'%Y-%m') as varchar(7)) as season,
        cast(case when month(cast(d.date_of_game as date)) <= 3 then concat(year(cast(d.date_of_game as date)),'_01')
            when month(cast(d.date_of_game as date)) <= 6 then concat(year(cast(d.date_of_game as date)),'_02')
            when month(cast(d.date_of_game as date)) <= 9 then concat(year(cast(d.date_of_game as date)),'_03')
            else concat(year(cast(d.date_of_game as date)),'_04') end as varchar(7))  as year_quarter,
        case when month(cast(d.date_of_game as date)) <= 3 then '01'
            when month(cast(d.date_of_game as date)) <= 6 then '02'
            when month(cast(d.date_of_game as date)) <= 9 then '03' else'04' end as quarter,
        weekofyear(cast(d.date_of_game as date)) as week_of_year,
        dayofweek(cast(d.date_of_game as date)) as day_of_week,
        case when dayofweekiso(cast(d.date_of_game as date)) in (6,7) then 1 else 0 end as weekend    
    from pz_clear_strategy.football_data d
    where d.date_of_game is not null
    order by cast(d.date_of_game as date)
"""

df_dim_area_of_shot = session.sql(query_dim_calendar)
df_dim_area_of_shot.write.mode("overwrite").saveAsTable('dim_calendar')


## DIM_AREA_OF_SHOT

In [65]:
table_name = 'dim_area_of_shot'

query_new = """select distinct    
                    upper(d.area_of_shot) as area_of_shot,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.area_of_shot is not null"""

query_append = """select distinct 
                        upper(d.area_of_shot) as area_of_shot,
                        sysdate() as load_at
                    from pz_clear_strategy.football_data d
                    where not exists (select 1 from hz_clear_strategy.dim_area_of_shot aos
                                            where aos.area_of_shot = upper(d.area_of_shot))
                    and d.area_of_shot is not null"""

create_hz_dim(table_name, query_new, query_append)

Table: dim_area_of_shot will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|SEQ_DIM_AREA_OF_SHOT already exists, statement ...  |
------------------------------------------------------

Execution finished, dim_area_of_shot created


## DIM_SHOT_BASICS

In [66]:
table_name = 'dim_shot_basics'

query_new = """select distinct    
                    d.shot_basics,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.shot_basics is not null"""

query_append = """select distinct    
                        UPPER(d.shot_basics) AS shot_basics,
                        sysdate() as load_at
                    from pz_clear_strategy.football_data d
                    where d.shot_basics is not null
                    and not exists (select 1 from hz_clear_strategy.dim_shot_basics dsb
                                            where dsb.shot_basics = UPPER(d.shot_basics))"""

create_hz_dim(table_name, query_new, query_append)

Table: dim_shot_basics will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|SEQ_DIM_SHOT_BASICS already exists, statement s...  |
------------------------------------------------------

Execution finished, dim_shot_basics created


## DIM_COMBINED_SHOT_TYPES

In [67]:
table_name = 'dim_combined_shot_types'

query_new = """select distinct
                upper(d.type_of_combined_shot) as combined_shot_type,
                sysdate() as load_at
            from pz_clear_strategy.football_data d
            where d.type_of_combined_shot is not null"""

query_append = """select distinct
                    upper(d.type_of_combined_shot) as combined_shot_type,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.type_of_combined_shot is not null
                and not exists (select 1 from hz_clear_strategy.dim_combined_shot_types cst
                                        where cst.combined_shot_type = UPPER(d.type_of_combined_shot))
"""
create_hz_dim(table_name, query_new, query_append)

------------------------------------------------------------------------------------
|"ID_DIM_COMBINED_SHOT_TYPES"  |"COMBINED_SHOT_TYPE"  |"LOAD_AT"                   |
------------------------------------------------------------------------------------
|115                           |SHOT - 30             |2023-06-10 18:42:56.202000  |
------------------------------------------------------------------------------------

Table: dim_combined_shot_types alread exists
Execution finished, dim_combined_shot_types appended


## DIM_TEAM

In [77]:
table_name = 'dim_team'

query_new = """select distinct
                    cast(d.team_id as int) as team_id,
                    UPPER(d.team_name) as team_name,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.team_name is not null"""

query_append = """select distinct
                    cast(d.team_id as int) as team_id,
                    UPPER(d.team_name) as team_name,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.type_of_combined_shot is not null
                and not exists (select 1 from hz_clear_strategy.dim_team dteam
                                        where dteam.team_name = UPPER(d.team_name))
"""
create_hz_dim(table_name, query_new, query_append)

Table: dim_team will be created
-----------------------------------------------
|"status"                                     |
-----------------------------------------------
|Sequence SEQ_DIM_TEAM successfully created.  |
-----------------------------------------------

Execution finished, dim_team created


In [78]:
table_name = 'dim_team'

query_new = """select distinct
                    cast(d.team_id as int) as team_id,
                    UPPER(d.team_name) as team_name,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.team_name is not null"""

query_append = """select distinct
                    null as team_id,
                    upper(trim(regexp_replace("home/away",'^([^ ]+ [^ ]+)',''))) as team_name,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where "home/away" is not null
                and not exists (select 1 from hz_clear_strategy.dim_team dteam
                                        where dteam.team_name = upper(trim(regexp_replace("home/away",'^([^ ]+ [^ ]+)',''))))
"""
create_hz_dim(table_name, query_new, query_append)

-------------------------------------------------------------------------------
|"ID_DIM_TEAM"  |"TEAM_ID"   |"TEAM_NAME"        |"LOAD_AT"                   |
-------------------------------------------------------------------------------
|1              |1610612747  |MANCHESTER UNITED  |2023-06-10 20:21:32.318000  |
-------------------------------------------------------------------------------

Table: dim_team alread exists
Execution finished, dim_team appended


## DIM_RANGE_OF_SHOT

In [79]:
table_name = 'dim_range_of_shot'

query_new = """select distinct    
                    upper(d.range_of_shot) as range_of_shot,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.range_of_shot is not null"""

query_append = """select distinct    
                    upper(d.range_of_shot) as range_of_shot,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.range_of_shot is not null
                and not exists (select 1 from hz_clear_strategy.dim_range_of_shot ros
                                        where ros.range_of_shot = UPPER(d.range_of_shot))
"""
create_hz_dim(table_name, query_new, query_append)

--------------------------------------------------------
|"ID_DIM_RANGE_OF_SHOT"  |"RANGE_OF_SHOT"  |"LOAD_AT"  |
--------------------------------------------------------
|                        |                 |           |
--------------------------------------------------------

Table: dim_range_of_shot alread exists
Table: dim_range_of_shot will be created
------------------------------------------------------
|"status"                                            |
------------------------------------------------------
|Sequence SEQ_DIM_RANGE_OF_SHOT successfully cre...  |
------------------------------------------------------

Execution finished, dim_range_of_shot created


## DIM_SHOT_TYPES

In [81]:
table_name = 'dim_shot_types'

query_new = """select distinct
                    upper(d.type_of_shot) as shot_type,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.type_of_shot is not null"""

query_append = """select distinct    
                    upper(d.type_of_shot) as shot_type,
                    sysdate() as load_at
                from pz_clear_strategy.football_data d
                where d.type_of_shot is not null
                and not exists (select 1 from hz_clear_strategy.dim_shot_types st
                                        where st.type_of_shot = UPPER(d.type_of_shot))
"""
create_hz_dim(table_name, query_new, query_append)

Table: dim_shot_types will be created
-----------------------------------------------------
|"status"                                           |
-----------------------------------------------------
|Sequence SEQ_DIM_SHOT_TYPES successfully created.  |
-----------------------------------------------------

Execution finished, dim_shot_types created
